## INTRODUCCIÓN

PCA es un técnica de **reducción de dimensión linear** que puede utilizarse para extraer información de desde un espacio dimensional superior a otro espacio dimensional inferior. Lo que buscamos con el PCA es preservar las parte que tienen más variación entre los datos y eliminar las partes con menos variación.  

Las dimensiones no son más que características (variables) que representan los datos. Por ejemplo, una imagen de 28 X 28 tiene 784 elementos de imagen (píxeles) que son las dimensiones o variables que juntas representan esa imagen.

Una cosa importante a tener en cuenta sobre PCA es que es una técnica de reducción de dimensionalidad **no supervisada**, donde se pueden grupar los puntos de datos similares en función de la correlación que tienen las variables sin ninguna supervisión (los datos no necesitan estar etiquetados)


Nota: Cuando hablamos de **características, dimensiones y variables** nos referimos a lo mismo

![texto alternativo](https://i.ibb.co/KwFjx8j/Capture-Data-FIFA20.png)

**Dónde se puede aplicar el PCA?**



*   **Visualización de Datos:**  Un paso importante en el proceso de exploración de los datos, es la visualización de datos. Especialmente en problemas como la correlación de variables o en entender la distribución de unas pocas variables. PCA Proyecta los datos en una dimensión inferior, permitiendo visualizar los datos en un espacio 2D o 3D en una simple pasada.

*   **Entrenamiento más rápido de modelos**: Ya que PCA se basa en la reducción de dimensiones, podemos aprovechar para acelerar el train y el test de los algoritmos de  ML, ya que los algoritmos con muchas dimensiones son lentos para entrenar 



### FIFA 20 DATASET

El siguiente dataset contiene más de 100 características de los jugadores del Videjuego FIFA 20. Este dataset pertenece a Kaggle y ha sido escrapeado de la web [sofifa.com](https://sofifa.com/). El contenido del dataset es el siguiente:



1.   Posición de los jugadores con el rol en el club y en su selección nacional.
2.   Atributos del jugador con estadísticas como Ataque, Habilidad, Defena, Mentalidad, Potencial, Pierna favorita, etc.,
3.   Datos personales del jugadores como Nacionalidad, Club, Fecha de Nacimiento, Salario, Valoración


Este notebook se va a centrar en  **Visualizar las habilidades de los jugadores**



In [ ]:
#Vamos a realizar una primera exploración de los datos. Para ello, vamos a importar algunos

### Vamos a realizar una primera exploración de los datos


In [1]:
from __future__ import division

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
%matplotlib inline

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix

Importamos el fichero con el que vamos a trabajar a Google Colab.

In [2]:
fifa20 = pd.read_csv("C:/Users/dario/OneDrive/Documentos/GitHub/M-ster-EDEM-22-23/3_Aprendizaje_Automatico/Machine_Learning/Regresión/Datasets/players_20.csv", index_col=0)

#Vamos a explorar el tamaño del dataset
fifa20.shape

(18278, 103)

In [3]:
#echamos un primer vistazo a los datos.

fifa20.head(10)

,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,overall,...,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
sofifa_id,,,,,,,,,,,,,,,,,,,,,
158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,32,1987-06-24,170,72,Argentina,FC Barcelona,94,...,68+2,66+2,66+2,66+2,68+2,63+2,52+2,52+2,52+2,63+2
20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,34,1985-02-05,187,83,Portugal,Juventus,93,...,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3
190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Junior,27,1992-02-05,175,68,Brazil,Paris Saint-Germain,92,...,66+3,61+3,61+3,61+3,66+3,61+3,46+3,46+3,46+3,61+3
200389,https://sofifa.com/player/200389/jan-oblak/20/...,J. Oblak,Jan Oblak,26,1993-01-07,188,87,Slovenia,Atlético Madrid,91,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
183277,https://sofifa.com/player/183277/eden-hazard/2...,E. Hazard,Eden Hazard,28,1991-01-07,175,74,Belgium,Real Madrid,91,...,66+3,63+3,63+3,63+3,66+3,61+3,49+3,49+3,49+3,61+3
192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,28,1991-06-28,181,70,Belgium,Manchester City,91,...,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3
192448,https://sofifa.com/player/192448/marc-andre-te...,M. ter Stegen,Marc-André ter Stegen,27,1992-04-30,187,85,Germany,FC Barcelona,90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
203376,https://sofifa.com/player/203376/virgil-van-di...,V. van Dijk,Virgil van Dijk,27,1991-07-08,193,92,Netherlands,Liverpool,90,...,79+3,83+3,83+3,83+3,79+3,81+3,87+3,87+3,87+3,81+3
177003,https://sofifa.com/player/177003/luka-modric/2...,L. Modrić,Luka Modrić,33,1985-09-09,172,66,Croatia,Real Madrid,90,...,81+3,81+3,81+3,81+3,81+3,79+3,72+3,72+3,72+3,79+3


Tenemos más de 100 Variables, por lo que es Normal pensar que muchas de ellas son redundantes.

## Correlación de Pearson

Una forma de responder a esto es creando una matriz de correlación utilizando un algoritmo como el de pearson.

El coeficiente de correlación de pearson tiene valores entre -1 y 1


*   Un valor cercano a 0 implica una correlación débil (si es 0 no hay ninguna correlación)
*   Un valor cercano al 1 implica una correlación fuertemente positiva
*   Un valor cercano al -1 implica una correlación fuertemente negativa

In [ ]:
#Using Pearson Correlation
plt.figure(figsize=(18,16))
cor = fifa20.corr()
sns.heatmap(cor, vmin=-1, cmap='BrBG')
plt.show()

Vemos que hay una fuerte correlación negativa abajo y otra en el lado derecho de la matriz. Estas habilidades son las de los porteros (conducción de balón, manos, posición de portero, reflejos) y están correlacionadas de manera negativa con las de los delanteros.  

Podemos también observar como las habilidades de los porteros están fuertamente correlacionadas centre ellas.




En general, independientemente de si están relacionados con los Porteros o no , existen muchas habilidades correlacionadas. Esto significa que podríamos no necesitarlas todas o podemos expresar la misma información con un conjunto de variables menor.

Como hemos comentado anteriormente, nos interesan únicamente las variables que hacen referencia a las habilidades de los jugadores. Así que vamos a crear otro dataset que contenga únicamente las columnas que nos interesan. 

### EJERCICIO. 
**Crea un dataset nuevo llamado "fifa_skills" que contenga únicamente las variables de los jugadores.**

Pista: Estas variables van desde la columna *pace* hasta la columna goalkeeping_reflexes

In [ ]:
#Nos interesan las solamente las habilidades de los jugadores. Creamos un dataset Nuevo.

#Cogemos los nombres de las columnas
fifa_skills =

In [ ]:
#Vamos a ver los tipos, ya que nos interesa que todas las columnas sean numéricas para poder representar los datos.
fifa_skills.dtypes

In [ ]:
#Hay una variable que es un objeto. Vamos a ver qué valores contiene.
fifa_skills['player_traits'].values

In [ ]:
#Se tratan de rasgos del jugadores. Eliminamos la columna, aunque otra opción sería convertirla a one-hot vectors.
fifa_skills = fifa_skills.drop("player_traits", axis=1)


###EJERCICIO 

Crea otra matriz de correlación de pearson. Utilizar ahora el mapa de color llamado *coolwarm* 




**¿QUÉ VARIABLES CREES TÚ QUE PODRÍAMOS ELIMINAR?**


In [ ]:
fifa_skills.columns.values

Para hacer el PCA, primero tenemos que lidiar con los missing values y los blank values.

---



In [ ]:
#Vamos a ver cuantos valores  tenemos para el PCA

fifa_skills.isnull().sum()[fifa_skills.isnull().sum() > 0]


Analizando las variables, vemos que tenemos variables duplicadas.

Eliminamos las variables gk_diving, gk_handling, gk_kicking, gk_reflexes, gk_speed y gk_positioning. Existe una misma variable para cada una de ellas que empieza por goalkipping_xxx







In [ ]:
columns_to_delete = ['gk_diving', 'gk_diving', 'gk_kicking', 'gk_reflexes', 'gk_speed', 'gk_positioning', 'gk_handling']

fifa_skills = fifa_skills.drop(columns_to_delete, axis=1)

fifa_skills.columns.values

### EJERCICIO

Aisla los null value y mira qué características tienen todos en común.

Pista: utiliza el atributo .isnull()

In [ ]:
#Vamos a estudiar qué características tienes los jugadores con Null Value




Vemos que todos los jugadores que tienen null values, tienen una características bastante significativa en común. Como no queremos deshacernos de ellos en el dataset, ponemos los atributos Null a 0



In [ ]:
fifa_skills.fillna(0, inplace=True)
fifa_skills.isnull().sum().sum()


In [ ]:
#Le añadimos el promedio de los jugadores. Nos servirá luego para quedarnos con los 100 mejores

fifa_skills = fifa_skills.join(fifa20['overall'])



Vamos a realizar un diagrama que nos indique el número de componentes que nos explican la variabilidad en los datos.ç

In [ ]:
#Escalamos los datos para hacer el diagrama que explica los componentes

from sklearn.preprocessing import MinMaxScaler


scaler = MinMaxScaler(feature_range=[-1, 1]) 
fifa_rescaled = scaler.fit_transform(fifa_skills)

In [ ]:
fifa_rescaled

In [ ]:
#Ajustando el PCA con nuestros datos
pca = PCA().fit(fifa_rescaled)
#Gráfica de la suma acumulativa de la varianza explicada
plt.figure()
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Número de Componentes')
plt.ylabel('Varianza (%)') #para cada componente
plt.title('FIFA 20 Dataset Explained Variance')
plt.show()

El gráfico de Arriba nos está diciendo que seleccionando **únicamente 10 componentes podemos mantener el 95% de la varianza en los datos**. Con 20 podríamos explicar en un 98% el comportamiento de los datos y con 25 podríamos explicar el 100%. 

Como hemos supuesto al principio, tenemos bastantes variables que son redundantes





## TOP 100 FIFA PLAYERS

Para mostrar algunos gráficos, vamos a quedarnos únicamente con el TOP 100 de Jugadores. 

In [ ]:
#Vamos a unir el nombre y las posiciones de los jugadores. Luego cogeremos el Top 100

fifa_skills = fifa_skills.join(fifa20[['short_name', 'player_positions']])
fifa_skills.head()


In [ ]:
fifa_top100 = fifa_skills.sort_values('overall', ascending=False).head(100).reset_index(drop=True).copy()


In [ ]:
#Comprobamos de nuevo si existen valores nulos
fifa_top100.isnull().sum().sum()

Vamos a utilizar una gráfica en 2 dimensiones para intentar explicar mejor los datos. Para ello utilizaremos PCA.

In [ ]:
X = fifa_top100.loc[:, 'pace':'overall'].copy()

#Nos quedaremos con 2 variables, para poder 

pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('decomposition', PCA(n_components=2))
    ])

In [ ]:
X_transformed = pipe.fit_transform(X)
X_transformed.shape

In [ ]:
# Vamos a ver el porcentaje de varianza que tiene cada uno de los componentes
pipe.named_steps['decomposition'].explained_variance_ratio_

In [ ]:
#Vamos a utilizar estos dos componentes para hacer un plot con un gráfico.

players_pca = pd.DataFrame(X_transformed, columns=['first_component', 'second_component'])



players_pca = players_pca.join(fifa_top100['short_name'])

In [ ]:
labels = players_pca['short_name'].head(30)
labels

In [ ]:
##Plot the first 30 elements

fig, ax = plt.subplots(1, 1, figsize=(15, 15))

ax.scatter(players_pca.first_component, players_pca.second_component, s=600, alpha=0.5)

ax.set_title('PCA Separa los porteros de los otros jugadores', size=20)
ax.set_xlabel('primer componente', size=16)
ax.set_ylabel('segundo componente', size=16)
ax.tick_params(labelsize=14)
ax.set_xlim(-6, 15)
ax.grid(False)

for label, x, y in zip(labels,
                       players_pca.first_component,
                       players_pca.second_component):
    ax.annotate(label, xy=(x, y), xytext=(-10, 5),
               textcoords='offset points', ha='right', va='bottom')
    
fig.tight_layout()

Con PCA, vamos a intentar diferenciar entre los jugadores de campo y los jugadores de 

## EJERCICIO

Utilizando las componentes de arriba, intenta encontrar diferencia entre únicamente los jugadores de campo delanteros, centrocampistas y defensas

In [ ]:
labels = otros_jugadores['short_name'].head(35)



fig, ax = plt.subplots(1, 1, figsize=(15, 15))

ax.scatter(otros_jugadores.first_component, otros_jugadores.second_component, s=600,
           alpha=0.5, c='#00D48A')

ax.set_title('Sólo jugadores de campo', size=20)
ax.set_xlabel('Primer Componente', size=16)
ax.set_ylabel('Segundo Componente', size=16)
ax.tick_params(labelsize=14)
ax.grid(False)



for label, x, y in zip(labels,
                       otros_jugadores.first_component,
                       otros_jugadores.second_component):
    ax.annotate(label, xy=(x, y), xytext=(-10, 5),
               textcoords='offset points', ha='right', va='bottom')
    



ALgunas conclusiones que sacamos es que los defensas están bastante separados de los delanteros.


Los jugadores más ofensivos están en la esquina inferior izquierda y los jugadores más defensivos están en la esquina superior derecha. 


In [ ]:
from IPython.display import clear_output
clear_output()
